In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Pre-processing Stage

In [2]:
no = pd.read_csv('no_discount.csv')
all = pd.read_csv('all_discount.csv')
half = pd.read_csv('half_discount.csv')
prod = pd.read_excel('product.xlsx')

## Data Preparation

In [3]:
# Map user features to number
dataframes = {'all': all, 'half': half, 'no': no}
feature_cols = ['user_feature_1', 'user_feature_2', 'user_feature_3']
category_maps = {}

# 1. Build mapping for each feature across all dataframes
for col in feature_cols:
    # Collect unique values from all dataframes for this column
    unique_vals = pd.concat([df[col] for df in dataframes.values()]).unique()
    cat_map = {cat: i+1 for i, cat in enumerate(sorted(unique_vals))}
    category_maps[col] = cat_map

# 2. Apply the mapping to each dataframe
for df in dataframes.values():
    for col in feature_cols:
        df[col + '_num'] = df[col].map(category_maps[col])

# 3. Show mapping for each feature
for col in feature_cols:
    print(f"\nMapping for {col}:")
    for cat, idx in category_maps[col].items():
        print(f"{idx}: {cat}")


Mapping for user_feature_1:
1: Female
2: Male
3: Non-binary / third gender
4: Prefer not to say

Mapping for user_feature_2:
1: 1-2 years ago
2: 2-3 years ago
3: 3-4 yeas ago
4: Currently, I don't have a phone
5: Less than 1 year ago
6: More than 4 years ago

Mapping for user_feature_3:
1: Brand reputation
2: Design and appearance
3: Good price
4: Product quality


In [4]:
# Map choice to number (no discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $24 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $26 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $25 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $25 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $26 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $27 (MSRP: $27)": 6
}

# Apply the mapping
no['choice_num'] = no['choice'].map(choice_map)

In [5]:
# Map choice to number (all discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $4.8 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $5.2 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $5 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $5 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $5.2 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $5.4 (MSRP: $27)": 6
}

# Apply the mapping
all['choice_num'] = all['choice'].map(choice_map)

In [6]:
# Map choice to number (half discount)
# Define the mapping for 'choice'
choice_map = {
    "Do not purchase; save the $30 or use it for other expenses ($30 is roughly enough to cover two meals at a fast-food restaurant in the U.S.)": 0,
    "Color: Black; Style: Solid color; Weight: 2.66 pounds; Price: $4.8 (MSRP: $24)": 1,
    "Color: Black; Style: Gradient color; Weight: 2.66 pounds; Price: $26 (MSRP: $26)": 2,
    "Color: Dark blue; Style: Solid color; Weight: 2.61 pounds; Price: $25 (MSRP: $25)": 3,
    "Color: Dark blue; Style: Gradient color; Weight: 2.64 pounds; Price: $25 (MSRP: $25)": 4,
    "Color: Light blue; Style: Gradient color; Weight: 2.68 pounds; Price: $5.2 (MSRP: $26)": 5,
    "Color: White; Style: Solid color; Weight: 2.68 pounds; Price: $5.4 (MSRP: $27)": 6
}

# Apply the mapping
half['choice_num'] = half['choice'].map(choice_map)

In [7]:
# Normalization: user features
# Define the columns to scale
columns_to_scale = ['user_feature_1_num', 'user_feature_2_num', 'user_feature_3_num']

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
half[columns_to_scale] = scaler.fit_transform(half[columns_to_scale])

In [8]:
# Normalization: product features and price
# Define the columns to scale
columns_to_scale = [
    'product_feature_1', 'product_feature_2', 'product_feature_3', 'price'
]

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
prod[columns_to_scale] = scaler.fit_transform(prod[columns_to_scale])

In [9]:
# Set parameters
discount_percentage = 0.2
num_product = 6

In [10]:
# Check the number of valid data
num_row_no = len(no)
num_row_all = len(all)
num_row_half = len(half)
print(num_row_no)
print(num_row_all)
print(num_row_half)

291
278
266


# Analysis Stage

## True GTE

In [11]:
# All products are control (no discount)
# Merge the purchasing data with the product data
merged_no = no.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the total revenue
revenue_control = 0
revenue_control = merged_no['price'].sum() * 300/291

# All products are treated (all discount)
# Merge the purchasing data with the product data
merged_all = all.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the total revenue
revenue_treated = 0
revenue_treated = merged_all['price'].sum() * discount_percentage * 300/278

# Calculate the GTE
true = revenue_treated - revenue_control
print(true)

-68.6551789496238


## DIM Estimator

In [12]:
# Define the function to calculate the revenue
def calculate_total_revenue(df):
    revenue_treated = 0.0
    revenue_control = 0.0
    df['adjusted_price'] = df.apply(
        lambda row: row['price'] * discount_percentage if row['if_treated'] == 1 else row['price'], axis=1
    )
    for _, row in df.iterrows():
        if row['if_treated'] == 0:
            revenue_control += row['adjusted_price']
        elif row['if_treated'] == 1:
            revenue_treated += row['adjusted_price']
    return revenue_control, revenue_treated

# Merge the purchasing data with the product data
merged_half = half.merge(prod, left_on='choice_num', right_on='product_id', how='left')

# Calculate the revenue difference
revenue_control, revenue_treated = calculate_total_revenue(merged_half)

# Calculate the GTE
naive = (revenue_treated - revenue_control) * 600/266
naive
naive_pe = abs((naive - true)/true)
print(naive_pe)

0.11024849998145407


## MNL Estimator

In [13]:
# Product features (skip the first row)
feature_cols = ['product_feature_1', 'product_feature_2', 'product_feature_3']
X_product = torch.tensor(prod[feature_cols].iloc[1:].to_numpy(), dtype=torch.float)

# Price (skip first row)
price = torch.tensor(prod['price'].iloc[1:].to_numpy(), dtype=torch.float)

# Product randomization (skip first row)
prod_randomization = torch.tensor(prod['if_treated'].iloc[1:].to_numpy(), dtype=torch.bool)

# User features
user_num_cols = ['user_feature_1_num', 'user_feature_2_num', 'user_feature_3_num']
X_user = torch.tensor(merged_half[user_num_cols].to_numpy(),dtype=torch.float)

# User choice
choice = torch.tensor(
    merged_half['choice_num'].to_numpy(),
    dtype=torch.long
)

In [14]:
# Split the training and test set
X_user_train, X_user_test, choice_train, choice_test = train_test_split(
    X_user, choice, test_size=0.5, random_state=42
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_user_train = X_user_train.to(device)
X_user_test  = X_user_test.to(device)
X_product    = X_product.to(device)
price        = price.to(device)
prod_randomization = prod_randomization.to(device)

choice_train = choice_train.to(device)
choice_test  = choice_test.to(device)

# Validation set
X_user_train1, X_user_val, choice_train1, choice_val = train_test_split(X_user_train, choice_train,test_size=0.1,random_state=34)

### No features

In [15]:
class LinearMNLModel_No(nn.Module):
    def __init__(self):
        super(LinearMNLModel_No, self).__init__()
        # v is the same constant value for all products.
        self.v = nn.Parameter(torch.tensor(1.0))
        # gamma is the price sensitivity parameter.
        self.gamma = nn.Parameter(torch.tensor(1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N = x_user.shape[0]
        # Adjust prices for treated products (discounted)
        adjusted_price = torch.where(
            prod_randomization,
            price * discount_percentage,
            price
        )
        # Compute utilities: shape [M]
        utility = self.v + self.gamma * adjusted_price
        # Expand to shape [N, M]
        utility = utility.expand(N, -1)
        # Add outside option (utility = 0)
        zero_utilities = torch.zeros(N, 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)
        return utilities_with_outside

In [16]:
model = LinearMNLModel_No().to(device)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=0.01) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)

num_epochs = 5000
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()
    scheduler.step()

    # --- 4. Validation phase (验证阶段) ---
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 2.959160089492798, Validation Loss: 2.948221445083618
Epoch 101, Training Loss: 1.98223876953125, Validation Loss: 1.9728033542633057
Epoch 201, Training Loss: 1.458035945892334, Validation Loss: 1.446217656135559
Epoch 301, Training Loss: 1.2844624519348145, Validation Loss: 1.2660506963729858
Epoch 401, Training Loss: 1.2421146631240845, Validation Loss: 1.2156528234481812
Epoch 501, Training Loss: 1.2312737703323364, Validation Loss: 1.1968811750411987
Epoch 601, Training Loss: 1.2267866134643555, Validation Loss: 1.1849260330200195
Epoch 701, Training Loss: 1.223840594291687, Validation Loss: 1.1750608682632446
Epoch 801, Training Loss: 1.2216732501983643, Validation Loss: 1.1666038036346436
Epoch 901, Training Loss: 1.2201117277145386, Validation Loss: 1.1594539880752563
Epoch 1001, Training Loss: 1.2190316915512085, Validation Loss: 1.153525471687317
Epoch 1101, Training Loss: 1.2183150053024292, Validation Loss: 1.1486985683441162
Epoch 1201, Training Los

In [17]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266 # For comparison purpose
print(linear)
linear_no = abs((linear - true)/true)
print(linear_no)

Expected Revenue: $37.51
Expected Revenue: $8.90
-64.5298821585519
0.060087190131699574


### Only user features

In [18]:
class LinearMNLModel_UserOnly(nn.Module):
    def __init__(self, user_feature_dim):
        super(LinearMNLModel_UserOnly, self).__init__()
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user features
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)

        # Price adjustment
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )
        utility_price = adjusted_price * self.beta_price
        utility_price = utility_price.expand(N, M)

        total_utility = utility_user + utility_price

        # Outside option
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)
        return utilities_with_outside

In [19]:
# Model training
model = LinearMNLModel_UserOnly(user_feature_dim=X_user.shape[1]).to(device)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=0.01) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)

num_epochs = 5000
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()
    scheduler.step()

    # --- 4. Validation phase (验证阶段) ---
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 1.4125741720199585, Validation Loss: 1.3267217874526978
Epoch 101, Training Loss: 1.256240963935852, Validation Loss: 1.07330322265625
Epoch 201, Training Loss: 1.2294076681137085, Validation Loss: 1.0203304290771484
Epoch 301, Training Loss: 1.2185896635055542, Validation Loss: 1.0105253458023071
Epoch 401, Training Loss: 1.2144628763198853, Validation Loss: 1.0156253576278687
Epoch 501, Training Loss: 1.21306312084198, Validation Loss: 1.0224069356918335
Epoch 601, Training Loss: 1.2126542329788208, Validation Loss: 1.027262568473816
Epoch 701, Training Loss: 1.2125521898269653, Validation Loss: 1.0300724506378174
Epoch 801, Training Loss: 1.2125308513641357, Validation Loss: 1.031492829322815
Epoch 901, Training Loss: 1.2125270366668701, Validation Loss: 1.0321329832077026
Epoch 1001, Training Loss: 1.2125264406204224, Validation Loss: 1.0323917865753174
Epoch 1101, Training Loss: 1.2125263214111328, Validation Loss: 1.0324859619140625
Epoch 1201, Training Lo

In [20]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_user_only = abs((linear - true)/true)
print(linear_user_only)

Expected Revenue: $36.18
Expected Revenue: $9.32
-60.58626963679952
0.11752805012342765


### Only product features

In [21]:
class LinearMNLModel_ProductOnly(nn.Module):
    def __init__(self, product_feature_dim):
        super(LinearMNLModel_ProductOnly, self).__init__()
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))

    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand product features
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Price adjustment
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )
        utility_price = adjusted_price * self.beta_price
        utility_price = utility_price.expand(N, M)

        total_utility = utility_product + utility_price

        # Outside option
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)
        return utilities_with_outside

In [22]:
# Model training
model = LinearMNLModel_ProductOnly(product_feature_dim=X_product.shape[1]).to(device)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=0.01) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)

num_epochs = 5000
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()
    scheduler.step()

    # --- 4. Validation phase (验证阶段) ---
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 1.8592193126678467, Validation Loss: 1.9835050106048584
Epoch 101, Training Loss: 1.4283355474472046, Validation Loss: 1.403351902961731
Epoch 201, Training Loss: 1.2947416305541992, Validation Loss: 1.2860618829727173
Epoch 301, Training Loss: 1.2360093593597412, Validation Loss: 1.2426822185516357
Epoch 401, Training Loss: 1.209613561630249, Validation Loss: 1.2220277786254883
Epoch 501, Training Loss: 1.1973379850387573, Validation Loss: 1.2087411880493164
Epoch 601, Training Loss: 1.1914604902267456, Validation Loss: 1.198697805404663
Epoch 701, Training Loss: 1.1886146068572998, Validation Loss: 1.1908971071243286
Epoch 801, Training Loss: 1.1872509717941284, Validation Loss: 1.184983491897583
Epoch 901, Training Loss: 1.18661630153656, Validation Loss: 1.1806682348251343
Epoch 1001, Training Loss: 1.1863341331481934, Validation Loss: 1.1776469945907593
Epoch 1101, Training Loss: 1.1862154006958008, Validation Loss: 1.1756161451339722
Epoch 1201, Training L

In [23]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_product_only = abs((linear - true)/true)
print(linear_product_only)

Expected Revenue: $24.67
Expected Revenue: $17.25
-16.727557935212786
0.7563540261472955


### All features

In [24]:
# MNL choice model
class LinearMNLModel(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(LinearMNLModel, self).__init__()
        # Initialize parameters for user and product features
        self.beta_user = nn.Parameter(torch.randn(user_feature_dim))
        self.beta_product = nn.Parameter(torch.randn(product_feature_dim))
        self.beta_price = nn.Parameter(torch.tensor(-1.0))
    def forward(self, x_user, X_product, price, prod_randomization):
        N, M = x_user.shape[0], X_product.shape[0]

        # Expand user and product features to create a [N, M, F] shaped tensor for each
        x_user_expanded = x_user.unsqueeze(1).expand(-1, M, -1).detach()
        X_product_expanded = X_product.unsqueeze(0).expand(N, -1, -1).detach()

        # Calculate linear utility from features
        utility_user = torch.sum(x_user_expanded * self.beta_user, dim=2)
        utility_product = torch.sum(X_product_expanded * self.beta_product, dim=2)

        # Adjust prices based on randomization
        adjusted_price = torch.where(
            prod_randomization.unsqueeze(0),
            price * discount_percentage,
            price
        )

        # Calculate utility from price, properly expanding its dimension
        utility_price = adjusted_price * self.beta_price  # [M]
        utility_price = utility_price.expand(N, M)  # [N, M]

        # Total utility including features and price
        total_utility = utility_user + utility_product + utility_price

        # Incorporate the outside option with utility 0
        zero_utilities = torch.zeros(N, 1, device=total_utility.device)
        utilities_with_outside = torch.cat((zero_utilities,total_utility), dim=1)

        return utilities_with_outside

In [25]:
# Model training
model = LinearMNLModel(
    user_feature_dim=X_user.shape[1],
    product_feature_dim=X_product.shape[1]
).to(device)
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=0.01) 
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)

num_epochs = 5000
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(num_epochs):
    # Set model to training mode
    model.train()
    optimizer.zero_grad()

    # Forward pass (Training)
    utilities = model(X_user_train1, X_product, price, prod_randomization)
    
    choice_probabilities = F.log_softmax(utilities, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])

    # L2 Regularization
    l2_norm = sum(param.pow(2.0).sum() for param in model.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward pass
    loss.backward()
    optimizer.step()
    scheduler.step()

    # --- 4. Validation phase (验证阶段) ---
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_utilities = model(X_user_val, X_product, price, prod_randomization)
        
        val_utilities = val_utilities - val_utilities.max(dim=1, keepdim=True).values
        
        val_choice_probabilities = F.log_softmax(val_utilities, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])

    if epoch % 100 == 0: 
        print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")

    if (val_loss < best_val_loss) | (val_loss < loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter
        torch.save(model.state_dict(), 'best_model_linear_mnl.pth')  # Save best model
    else:
        patience_counter += 1  

    if patience_counter >= patience:
        print(f"Early stopping triggered at epoch {epoch}")
        break

Epoch 1, Training Loss: 2.5558688640594482, Validation Loss: 2.8942508697509766
Epoch 101, Training Loss: 1.3533287048339844, Validation Loss: 1.249069094657898
Epoch 201, Training Loss: 1.2533549070358276, Validation Loss: 1.0864800214767456
Epoch 301, Training Loss: 1.2135553359985352, Validation Loss: 1.065589189529419
Epoch 401, Training Loss: 1.1913317441940308, Validation Loss: 1.0661247968673706
Epoch 501, Training Loss: 1.1781517267227173, Validation Loss: 1.075846552848816
Epoch 601, Training Loss: 1.170103669166565, Validation Loss: 1.0886338949203491
Epoch 701, Training Loss: 1.1652265787124634, Validation Loss: 1.1014353036880493
Epoch 801, Training Loss: 1.1623575687408447, Validation Loss: 1.1129199266433716
Epoch 901, Training Loss: 1.1607362031936646, Validation Loss: 1.1226067543029785
Epoch 1001, Training Loss: 1.159858226776123, Validation Loss: 1.1304233074188232
Epoch 1101, Training Loss: 1.159401774406433, Validation Loss: 1.1365025043487549
Epoch 1201, Training L

In [26]:
# All control and all treated in test set
all_product_control = np.random.choice([0,1], num_product, p=[1, 0])
all_product_treated = np.random.choice([0,1], num_product, p=[0, 1])
all_product_control = torch.from_numpy(all_product_control).to(X_user_train.device).bool()
all_product_treated = torch.from_numpy(all_product_treated).to(X_user_train.device).bool()
X_user_test, X_product, price = X_user_test.to(device), X_product.to(device), price.to(device)

# Calculate expected revenue if all control
utilities = model(X_user_test, X_product, price, all_product_control)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum()
print(f"Expected Revenue: ${expected_revenue.item():.2f}")

# Calculate expected revenue if all treated
utilities = model(X_user_test, X_product, price, all_product_treated)
probabilities = F.softmax(utilities, dim=1)  # Convert utilities to probabilities
price_with_outside = torch.cat((torch.zeros(1, device=price.device),price), dim=0)
expected_revenue_treated = torch.sum(probabilities * price_with_outside.unsqueeze(0).expand_as(probabilities), dim=0).sum() * discount_percentage
print(f"Expected Revenue: ${expected_revenue_treated.item():.2f}")

# Calculate GTE
linear = (expected_revenue_treated-expected_revenue).cpu().detach().numpy()
linear = linear * 600 / 266
print(linear)
linear_all = abs((linear - true)/true)
print(linear_all)

Expected Revenue: $26.00
Expected Revenue: $13.44
-28.316716502483626
0.5875516321461894


## PDL Estimator

In [27]:
# Prepare data
def prepare_data(user_features, product_features, prices):
    num_products = product_features.shape[0]
    all_x_other_products = []
    all_other_prices = []
    for i in range(num_products):
        indices = [j for j in range(num_products) if j != i]
        other_products = product_features[indices].reshape(-1)
        other_product_prices = prices[indices]
        all_x_other_products.append(other_products)
        all_other_prices.append(other_product_prices)
    # Convert lists to tensor
    all_x_other_products = torch.stack(all_x_other_products, dim=0)
    all_other_prices = torch.stack(all_other_prices, dim=0)
    return user_features, product_features, prices, all_x_other_products, all_other_prices

### No features

In [28]:
class DeepMNLModel_No(nn.Module):
    """
    Price-only Deep MNL:
    u_ij = theta(p_j)
    Outside option utility = 0.
    """
    def __init__(self, num_product, hidden=5):
        super(DeepMNLModel_No, self).__init__()

        # theta(p_j):
        # Input:  [1]
        # Output: scalar utility
        self.theta = nn.Sequential(
        nn.Linear(1, hidden),
        nn.ReLU(),
        nn.Linear(hidden, 1)

)

    def forward(self, x_user, X_product, X_other_products, x_other_prices, price, prod_randomization):
        """
        Inputs used:
        - price: [M]  own price p_j
        - prod_randomization: [M]  treatment indicator
        """
        device = price.device
        N, M = x_user.shape[0], price.shape[0]

        # Apply treatment discount to own prices
        adjusted_price = torch.where(
            prod_randomization.to(device).bool(),
            price * discount_percentage,
            price
        )  # [M]

        # Prepare input for theta: [M, 1]
        pj_input = adjusted_price.unsqueeze(1)

        # Expand to all users: [N, M, 1]
        theta_input_exp = pj_input.unsqueeze(0).expand(N, -1, -1)

        # u_ij = theta(p_j): [N, M]
        utilities = self.theta(theta_input_exp).squeeze(-1)

        # Outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [29]:
# Apply the model
pdlmodel = DeepMNLModel_No(
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Weight initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_No(
  (theta): Sequential(
    (0): Linear(in_features=1, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [30]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [31]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 10
patience_counter = 0

# ----- Training loop -----
for epoch in range(5000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )
    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 1.8457, Validation Loss: 1.8544
Epoch 2, Training Loss: 1.8276, Validation Loss: 1.8375
Epoch 3, Training Loss: 1.8097, Validation Loss: 1.8208
Epoch 4, Training Loss: 1.7920, Validation Loss: 1.8045
Epoch 5, Training Loss: 1.7746, Validation Loss: 1.7887
Epoch 6, Training Loss: 1.7578, Validation Loss: 1.7739
Epoch 7, Training Loss: 1.7417, Validation Loss: 1.7592
Epoch 8, Training Loss: 1.7259, Validation Loss: 1.7448
Epoch 9, Training Loss: 1.7103, Validation Loss: 1.7306
Epoch 10, Training Loss: 1.6949, Validation Loss: 1.7166
Epoch 11, Training Loss: 1.6798, Validation Loss: 1.7028
Epoch 12, Training Loss: 1.6650, Validation Loss: 1.6892
Epoch 13, Training Loss: 1.6503, Validation Loss: 1.6758
Epoch 14, Training Loss: 1.6360, Validation Loss: 1.6627
Epoch 15, Training Loss: 1.6218, Validation Loss: 1.6497
Epoch 16, Training Loss: 1.6080, Validation Loss: 1.6370
Epoch 17, Training Loss: 1.5944, Validation Loss: 1.6245
Epoch 18, Training Loss: 1.5810, Validat

In [32]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_no = abs((pdl - true) / true)
print(pdl_no)


Expected Revenue (all control): $37.55
Expected Revenue (all treated): $8.87
-64.70293747751336
0.05756654534409455


### Only user features

In [60]:
class DeepMNLModel_UserOnly(nn.Module):
    """
    Deep MNL (user + own price only):
    u_ij = theta(x_i, p_j)
    Outside option utility = 0.
    """
    def __init__(self, user_dim, num_product, hidden=5):
        super(DeepMNLModel_UserOnly, self).__init__()

        # theta(x_i, p_j)
        # Input per (i,j): [user_dim + 1]
        self.theta = nn.Sequential(
            nn.Linear(user_dim + 1, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):

        device = price.device
        N, M = x_user.shape[0], price.shape[0]

        # p_j: [M, 1]
        pj_feat = price.unsqueeze(1)          # [M, 1]

        pj_feat_exp = pj_feat.unsqueeze(0).expand(N, -1, -1)

        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)

        theta_input = torch.cat([x_user_exp, pj_feat_exp], dim=2)

        theta_input_flat = theta_input.reshape(N * M, -1)

        # u_ij: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside


In [61]:
# Apply the model
pdlmodel = DeepMNLModel_UserOnly(
    user_dim=X_user.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Weight initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_UserOnly(
  (theta): Sequential(
    (0): Linear(in_features=4, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [62]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [63]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 10
patience_counter = 0

# ----- Training loop -----
for epoch in range(5000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )
    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 1.9507, Validation Loss: 1.9826
Epoch 2, Training Loss: 1.9270, Validation Loss: 1.9560
Epoch 3, Training Loss: 1.9032, Validation Loss: 1.9297
Epoch 4, Training Loss: 1.8794, Validation Loss: 1.9033
Epoch 5, Training Loss: 1.8558, Validation Loss: 1.8767
Epoch 6, Training Loss: 1.8321, Validation Loss: 1.8495
Epoch 7, Training Loss: 1.8084, Validation Loss: 1.8223
Epoch 8, Training Loss: 1.7849, Validation Loss: 1.7955
Epoch 9, Training Loss: 1.7615, Validation Loss: 1.7682
Epoch 10, Training Loss: 1.7377, Validation Loss: 1.7411
Epoch 11, Training Loss: 1.7145, Validation Loss: 1.7142
Epoch 12, Training Loss: 1.6912, Validation Loss: 1.6871
Epoch 13, Training Loss: 1.6676, Validation Loss: 1.6602
Epoch 14, Training Loss: 1.6443, Validation Loss: 1.6325
Epoch 15, Training Loss: 1.6207, Validation Loss: 1.6038
Epoch 16, Training Loss: 1.5965, Validation Loss: 1.5740
Epoch 17, Training Loss: 1.5720, Validation Loss: 1.5439
Epoch 18, Training Loss: 1.5472, Validat

In [64]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_user = abs((pdl - true) / true)
print(pdl_user)

Expected Revenue (all control): $36.02
Expected Revenue (all treated): $8.55
-61.969047202203505
0.09738714325289709


### Only product features

In [65]:
class DeepMNLModel_ProductOnly(nn.Module):
    """
    Deep MNL (product-only):
    u_ij = theta(z_j, z_-j, p_j)
    Outside option utility = 0.
    """
    def __init__(self, product_dim, num_product, hidden=5):
        super(DeepMNLModel_ProductOnly, self).__init__()

        # Aggregate other products' features z_-j:
        # Input:  [M, (M-1)*F_p]
        # Output: [M, hidden]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_dim * (num_product - 1), hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU()
        )

        # theta(z_j, z_-j, p_j)
        # Input per (i,j): [F_p + hidden (z_-j) + 1 (p_j)]
        in_dim = product_dim + hidden + 1

        self.theta = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):
        """
        X_product:        [M, F_p]          product features z_j
        X_other_products: [M, (M-1)*F_p]    other products' features z_-j
        price:            [M]               own price p_j (already discounted)
        """
        device = price.device
        # N users, M products
        N, M = x_user.shape[0], X_product.shape[0]

        # z_-j features: [M, hidden]
        zminus_feat = self.other_product_features_layers(X_other_products)

        # p_j scalar: [M, 1]
        pj_feat = price.unsqueeze(1)

        # Combine product-side info: [M, F_p + hidden + 1]
        product_feat = torch.cat(
            [X_product, zminus_feat, pj_feat],
            dim=1
        )

        # Expand to all users: [N, M, in_dim]
        product_feat_exp = product_feat.unsqueeze(0).expand(N, -1, -1)

        # Flatten to [N*M, in_dim]
        theta_input_flat = product_feat_exp.reshape(N * M, -1)

        # Utilities: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # Outside option = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [66]:
# Apply the model
pdlmodel = DeepMNLModel_ProductOnly(
    product_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Weight initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_ProductOnly(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
  )
  (theta): Sequential(
    (0): Linear(in_features=9, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [67]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [68]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0  # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 10
patience_counter = 0

# ----- Training loop -----
for epoch in range(5000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )

    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 1.6353, Validation Loss: 1.4547
Epoch 2, Training Loss: 1.5510, Validation Loss: 1.3451
Epoch 3, Training Loss: 1.4792, Validation Loss: 1.2545
Epoch 4, Training Loss: 1.4136, Validation Loss: 1.1768
Epoch 5, Training Loss: 1.3504, Validation Loss: 1.1154
Epoch 6, Training Loss: 1.3049, Validation Loss: 1.0782
Epoch 7, Training Loss: 1.2843, Validation Loss: 1.0652
Epoch 8, Training Loss: 1.2892, Validation Loss: 1.0689
Epoch 9, Training Loss: 1.3046, Validation Loss: 1.0748
Epoch 10, Training Loss: 1.3134, Validation Loss: 1.0756
Epoch 11, Training Loss: 1.3098, Validation Loss: 1.0711
Epoch 12, Training Loss: 1.2960, Validation Loss: 1.0646
Epoch 13, Training Loss: 1.2767, Validation Loss: 1.0599
Epoch 14, Training Loss: 1.2572, Validation Loss: 1.0587
Epoch 15, Training Loss: 1.2422, Validation Loss: 1.0628
Epoch 16, Training Loss: 1.2323, Validation Loss: 1.0718
Epoch 17, Training Loss: 1.2276, Validation Loss: 1.0838
Epoch 18, Training Loss: 1.2269, Validat

In [69]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_product = abs((pdl - true) / true)
print(pdl_product)

Expected Revenue (all control): $39.86
Expected Revenue (all treated): $10.71
-65.75381917164738
0.04225988224581449


### All features

In [80]:
# PDL choice model
class DeepMNLModel_UserProduct(nn.Module):
    """
    Deep MNL:
    u_ij = theta(x_i, z_j, z_-j, p_j)
    Outside option utility = 0.
    """
    def __init__(self, user_dim, product_dim, num_product, hidden=5):
        super(DeepMNLModel_UserProduct, self).__init__()

        # Aggregate other products' features z_-j:
        # Input:  [M, (M-1)*F_p]
        # Output: [M, hidden]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_dim * (num_product - 1), hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU()
        )

        # theta(x_i, z_j, z_-j, p_j)
        # Input per (i,j):
        #   user_dim (x_i)
        # + product_dim (z_j)
        # + hidden (z_-j)
        # + 1 (p_j)
        in_dim = user_dim + product_dim + hidden + 1

        self.theta = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, X_product, X_other_products,
                x_other_prices, price, prod_randomization):
        """
        x_user:          [N, F_u]          user features x_i
        X_product:       [M, F_p]          product features z_j
        X_other_products:[M, (M-1)*F_p]    other products' features z_-j
        price:           [M]               own price p_j (already discounted)

        x_other_prices, prod_randomization are not used here.
        """
        device = price.device
        N, M = x_user.shape[0], X_product.shape[0]

        # z_-j features: [M, hidden]
        zminus_feat = self.other_product_features_layers(X_other_products)

        # p_j: [M, 1]
        pj_feat = price.unsqueeze(1)

        # Product-side block: [M, F_p + hidden + 1]
        product_block = torch.cat(
            [X_product, zminus_feat, pj_feat],
            dim=1
        )

        # Expand product block for all users: [N, M, *]
        product_block_exp = product_block.unsqueeze(0).expand(N, -1, -1)

        # Expand user features: [N, 1, F_u] -> [N, M, F_u]
        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)

        # Final theta input: [N, M, in_dim]
        theta_input = torch.cat([x_user_exp, product_block_exp], dim=2)

        # Flatten to [N*M, in_dim]
        theta_input_flat = theta_input.reshape(N * M, -1)

        # Utilities: [N*M, 1] -> [N, M]
        utilities = self.theta(theta_input_flat).view(N, M)

        # Outside option = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)

        return utilities_with_outside

In [81]:
# Apply the model
pdlmodel = DeepMNLModel_UserProduct(
    user_dim=X_user.shape[1],
    product_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Weight initialization
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

pdlmodel.apply(init_weights)

DeepMNLModel_UserProduct(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
  )
  (theta): Sequential(
    (0): Linear(in_features=12, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=5, bias=True)
    (3): ReLU()
    (4): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [82]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [83]:
# ----- Prepare validation data -----
prepared_val = prepare_data(
    X_user_val,
    X_product,
    price * (1 - (1 - discount_percentage) * prod_randomization)
)
user_features_val, product_features_val, _, all_x_other_products_val, all_other_prices_val = prepared_val

user_features_val = user_features_val.to(device)
product_features_val = product_features_val.to(device)
all_x_other_products_val = all_x_other_products_val.to(device)
all_other_prices_val = all_other_prices_val.to(device)
choice_val = choice_val.to(device)

# ----- Optimizer and scheduler -----
optimizer = optim.Adam(pdlmodel.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0 # L2 regularization coefficient

# Early stopping setup
best_val_loss = float('inf')
patience = 10
patience_counter = 0

# ----- Training loop -----
for epoch in range(5000):
    pdlmodel.train()
    optimizer.zero_grad()

    # Forward pass (train): use prepared prices and other-prices matrix
    outputs = pdlmodel(
        user_features,         # x_user (not used inside DeepMNLModel_No, but kept for interface)
        product_features,      # X_product (not used)
        all_x_other_products,  # X_other_products (not used in No model)
        all_other_prices,      # x_other_prices: p_-j
        prices,                # price: experimental p_j (after discount)
        prod_randomization     # can be ignored inside No model
    )

    choice_probabilities = F.log_softmax(outputs, dim=1)

    # Training loss
    loss = -torch.mean(
        choice_probabilities[
            torch.arange(choice_probabilities.shape[0], device=device),
            choice_train1
        ]
    )

    # L2 regularization
    l2_norm = sum(param.pow(2.0).sum() for param in pdlmodel.parameters())
    loss = loss + l2_lambda * l2_norm

    # Backward
    loss.backward()
    optimizer.step()

    # ----- Validation -----
    pdlmodel.eval()
    with torch.no_grad():
        val_outputs = pdlmodel(
            user_features_val,
            product_features_val,
            all_x_other_products_val,
            all_other_prices_val,
            prices,              # same product prices for validation
            prod_randomization
        )
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(
            val_choice_probabilities[
                torch.arange(val_choice_probabilities.shape[0], device=device),
                choice_val
            ]
        )

    print(
        f"Epoch {epoch+1}, "
        f"Training Loss: {loss.item():.4f}, "
        f"Validation Loss: {val_loss.item():.4f}"
    )

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(pdlmodel.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    scheduler.step()

Epoch 1, Training Loss: 2.1402, Validation Loss: 2.0631
Epoch 2, Training Loss: 2.0515, Validation Loss: 2.0048
Epoch 3, Training Loss: 1.9912, Validation Loss: 1.9618
Epoch 4, Training Loss: 1.9509, Validation Loss: 1.9343
Epoch 5, Training Loss: 1.9328, Validation Loss: 1.9232
Epoch 6, Training Loss: 1.9249, Validation Loss: 1.9176
Epoch 7, Training Loss: 1.9197, Validation Loss: 1.9120
Epoch 8, Training Loss: 1.9145, Validation Loss: 1.9064
Epoch 9, Training Loss: 1.9093, Validation Loss: 1.9008
Epoch 10, Training Loss: 1.9041, Validation Loss: 1.8952
Epoch 11, Training Loss: 1.8990, Validation Loss: 1.8896
Epoch 12, Training Loss: 1.8938, Validation Loss: 1.8841
Epoch 13, Training Loss: 1.8887, Validation Loss: 1.8786
Epoch 14, Training Loss: 1.8836, Validation Loss: 1.8731
Epoch 15, Training Loss: 1.8785, Validation Loss: 1.8676
Epoch 16, Training Loss: 1.8734, Validation Loss: 1.8621
Epoch 17, Training Loss: 1.8684, Validation Loss: 1.8567
Epoch 18, Training Loss: 1.8633, Validat

In [84]:
# All-control and all-treated for test set
num_product = X_product.shape[0]

# All control: product_randomization = 0
all_product_control = torch.zeros(num_product, device=device).bool()

# All treated: product_randomization = 1
all_product_treated = torch.ones(num_product, device=device).bool()

# Move base price and test users/products to device
X_user_test = X_user_test.to(device)
X_product = X_product.to(device)
price = price.to(device)

# --------- Expected revenue: all control ----------
# Control prices = base prices (no discount)
price_control = price.clone()

# Prepare test data under all-control prices
user_test_ctrl, product_test_ctrl, prices_ctrl, x_other_products_ctrl, other_prices_ctrl = prepare_data(
    X_user_test,
    X_product,
    price_control
)

utilities_ctrl = pdlmodel(
    user_test_ctrl,
    product_test_ctrl,
    x_other_products_ctrl,
    other_prices_ctrl,
    prices_ctrl,          # control prices
    all_product_control   # all False
)

probabilities_ctrl = F.softmax(utilities_ctrl, dim=1)

price_with_outside_ctrl = torch.cat(
    (torch.zeros(1, device=price.device), prices_ctrl), dim=0
)  # [M+1]

expected_revenue_ctrl = torch.sum(
    probabilities_ctrl * price_with_outside_ctrl.unsqueeze(0).expand_as(probabilities_ctrl),
    dim=1
).sum()

print(f"Expected Revenue (all control): ${expected_revenue_ctrl.item():.2f}")

# --------- Expected revenue: all treated ----------
# Treated prices = discounted prices
price_treated = price * discount_percentage

user_test_trt, product_test_trt, prices_trt, x_other_products_trt, other_prices_trt = prepare_data(
    X_user_test,
    X_product,
    price_treated
)

utilities_trt = pdlmodel(
    user_test_trt,
    product_test_trt,
    x_other_products_trt,
    other_prices_trt,
    prices_trt,          # treated prices
    all_product_treated  # all True
)

probabilities_trt = F.softmax(utilities_trt, dim=1)

price_with_outside_trt = torch.cat(
    (torch.zeros(1, device=price.device), prices_trt), dim=0
)

expected_revenue_trt = torch.sum(
    probabilities_trt * price_with_outside_trt.unsqueeze(0).expand_as(probabilities_trt),
    dim=1
).sum()

print(f"Expected Revenue (all treated): ${expected_revenue_trt.item():.2f}")

# --------- GTE ----------
pdl = (expected_revenue_trt - expected_revenue_ctrl).cpu().detach().numpy() * 600 / 266
print(pdl)

pdl_all = abs((pdl - true) / true)
print(pdl_all)

Expected Revenue (all control): $41.17
Expected Revenue (all treated): $8.23
-74.286465178755
0.08202274490119944


## DML Estimator

In [48]:
def H_theta(theta0_output,theta1_output,all_treated_price,price):
    N = theta0_output.shape[0]
    M = num_product
    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = all_treated_price.unsqueeze(0).expand(N, M)
    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti =  theta0_output + theta1_output * expand_price

    zero_utilities = torch.zeros(N, 1, device=all_treated_uti.device)
    all_treated_uti = torch.cat((zero_utilities,all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities,all_control_uti), dim=1)

    all_treated_probabilities = F.softmax(all_treated_uti, dim=1)
    all_control_probabilities = F.softmax(all_control_uti, dim=1)

    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)
    treated_price_with_outside = torch.cat((torch.zeros(1, device=all_treated_price.device), all_treated_price), dim=0)

    H = torch.sum(all_treated_probabilities * treated_price_with_outside - all_control_probabilities * price_with_outside,dim=1)
    expsum_treated = torch.sum(torch.exp(all_treated_uti),dim=1)
    expsum_control = torch.sum(torch.exp(all_control_uti),dim=1)

    expsum_treated_expanded = expsum_treated.unsqueeze(1).expand(-1, all_treated_uti.shape[1])
    expsum_control_expanded = expsum_control.unsqueeze(1).expand(-1, all_control_uti.shape[1])

    H_theta0 = torch.sum((torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded) * treated_price_with_outside-
                          (torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded) * price_with_outside,dim=1)
    H_theta1 = torch.sum((torch.exp(all_treated_uti)*(1-torch.exp(all_treated_uti))/expsum_treated_expanded/expsum_treated_expanded) * treated_price_with_outside * treated_price_with_outside-
                          (torch.exp(all_control_uti)*(1-torch.exp(all_control_uti))/expsum_control_expanded/expsum_control_expanded) * price_with_outside * price_with_outside,dim=1)

    return H,H_theta0,H_theta1

In [49]:
# l_theta
def l_theta(theta0_output, theta1_output, adjusted_price, decision_test, l2_lambda=1e-4):
    N = theta0_output.shape[0]
    M = theta0_output.shape[1]

    expand_adjusted_price = adjusted_price.unsqueeze(0).expand(N, M)
    uti = theta0_output + theta1_output * expand_adjusted_price

    zero_utilities = torch.zeros(N, 1, device=uti.device)
    uti_with_outside = torch.cat((zero_utilities, uti), dim=1)

    probabilities = F.softmax(uti_with_outside, dim=1)

    prod_indices = torch.ones(M, device=uti.device)
    prod_indices = torch.cat([torch.zeros(1, device=uti.device), prod_indices])

    adjusted_price_with_outside = torch.cat([torch.zeros(1, device=adjusted_price.device), adjusted_price])

    ltheta0 = probabilities[torch.arange(decision_test.size(0)), decision_test] - prod_indices[decision_test]

    ltheta1 = (probabilities[torch.arange(decision_test.size(0)), decision_test] * adjusted_price_with_outside[decision_test]) - \
              (adjusted_price_with_outside[decision_test] * prod_indices[decision_test])

    is_inside = (decision_test > 0)

    inside_indices = (decision_test - 1).clamp(min=0)

    theta0_chosen = theta0_output.gather(1, inside_indices.unsqueeze(1)).squeeze(1)
    theta1_chosen = theta1_output.gather(1, inside_indices.unsqueeze(1)).squeeze(1)

    reg_grad0 = 2 * l2_lambda * theta0_chosen
    reg_grad1 = 2 * l2_lambda * theta1_chosen

    ltheta0 = ltheta0 + (reg_grad0 * is_inside.float())
    ltheta1 = ltheta1 + (reg_grad1 * is_inside.float())

    return ltheta0, ltheta1

In [50]:
# l_inv
def lambdainv(theta0_output, theta1_output, price, decision_test, epsilon=1e-6, l2_lambda=1e-4):

    N = theta0_output.shape[0]
    M = num_product

    expand_price = price.unsqueeze(0).expand(N, M)
    expand_all_treated_price = discount_percentage * price.unsqueeze(0).expand(N, M)

    all_treated_uti = theta0_output + theta1_output * expand_all_treated_price
    all_control_uti = theta0_output + theta1_output * expand_price

    zero_utilities = torch.zeros(N, 1, device=all_control_uti.device)
    all_treated_uti = torch.cat((zero_utilities, all_treated_uti), dim=1)
    all_control_uti = torch.cat((zero_utilities, all_control_uti), dim=1)

    probabilities_control = F.softmax(all_control_uti, dim=1)
    probabilities_treated = F.softmax(all_treated_uti, dim=1)

    chosen_prob_control = probabilities_control[torch.arange(N), decision_test]
    chosen_prob_treated = probabilities_treated[torch.arange(N), decision_test]

    price_with_outside = torch.cat((torch.zeros(1, device=price.device), price), dim=0)
    all_treated_price_vec = price * discount_percentage # Assuming logic from original code
    treated_price_with_outside = torch.cat((torch.zeros(1, device=price.device), all_treated_price_vec), dim=0)

    expand_price_ext = price_with_outside.unsqueeze(0).expand(N, M+1)

    chosen_price = expand_price_ext[torch.arange(N), decision_test]

    ltheta00 = chosen_prob_control * (1 - chosen_prob_control) + \
               chosen_prob_treated * (1 - chosen_prob_treated)

    ltheta01 = chosen_prob_control * (1 - chosen_prob_control) * chosen_price + \
               chosen_prob_treated * (1 - chosen_prob_treated) * (chosen_price * discount_percentage)

    ltheta11 = chosen_prob_control * (1 - chosen_prob_control) * (chosen_price**2) + \
               chosen_prob_treated * (1 - chosen_prob_treated) * ((chosen_price * discount_percentage)**2)

    ltheta00 = ltheta00 / 2
    ltheta01 = ltheta01 / 2
    ltheta11 = ltheta11 / 2

    ltheta00 = ltheta00.unsqueeze(1).unsqueeze(2)
    ltheta01 = ltheta01.unsqueeze(1).unsqueeze(2)
    ltheta11 = ltheta11.unsqueeze(1).unsqueeze(2)

    top_row = torch.cat((ltheta00, ltheta01), dim=2)
    bottom_row = torch.cat((ltheta01, ltheta11), dim=2)

    L_matrix = torch.cat((top_row, bottom_row), dim=1) # [N, 2, 2]

    reg_hessian_value = 2 * l2_lambda
    identity_matrix = torch.eye(2, dtype=L_matrix.dtype, device=L_matrix.device)
    reg_matrix = identity_matrix.unsqueeze(0).expand(N, 2, 2) * reg_hessian_value

    L_total = L_matrix + reg_matrix + (identity_matrix.unsqueeze(0) * epsilon)

    L_inv = torch.linalg.inv(L_total)

    return L_inv

### No features

In [51]:
class UtilityEstimator_No(nn.Module):
    def __init__(self, num_product): 
        super(UtilityEstimator_No, self).__init__()
        self.num_product = num_product

        self.theta0 = nn.Parameter(torch.tensor(1.0)) 
        self.theta1 = nn.Parameter(torch.tensor(-1.0)) 

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        price: [M]
        """
        N = x_user.shape[0]
        M = self.num_product

        u_product = self.theta0 + self.theta1 * price 
        
        utilities = u_product.unsqueeze(0).expand(N, -1)
        zero_utilities = torch.zeros(N, 1, device=price.device)
        utilities_with_outside = torch.cat([zero_utilities, utilities], dim=1)


        theta0_expanded = self.theta0.view(1, 1).expand(N, M)
        theta1_expanded = self.theta1.view(1, 1).expand(N, M)

        return utilities_with_outside, theta0_expanded, theta1_expanded

In [52]:
# Apply the model
dml_model = UtilityEstimator_No(
    num_product=X_product.shape[0]
).to(device)


# Initialize weight
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)
dml_model.apply(init_weights)

UtilityEstimator_No()

In [85]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [86]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(5000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 2.2338318824768066, Validation Loss: 2.309324026107788
Epoch 2, Training Loss: 2.2258245944976807, Validation Loss: 2.301340103149414
Epoch 3, Training Loss: 2.217845916748047, Validation Loss: 2.2933857440948486
Epoch 4, Training Loss: 2.20989727973938, Validation Loss: 2.28546142578125
Epoch 5, Training Loss: 2.2019786834716797, Validation Loss: 2.2775676250457764
Epoch 6, Training Loss: 2.1940908432006836, Validation Loss: 2.2697055339813232
Epoch 7, Training Loss: 2.186234474182129, Validation Loss: 2.2618746757507324
Epoch 8, Training Loss: 2.1784098148345947, Validation Loss: 2.2540760040283203
Epoch 9, Training Loss: 2.1706178188323975, Validation Loss: 2.2463104724884033
Epoch 10, Training Loss: 2.1628589630126953, Validation Loss: 2.2385780811309814
Epoch 11, Training Loss: 2.1551339626312256, Validation Loss: 2.230879306793213
Epoch 12, Training Loss: 2.1474428176879883, Validation Loss: 2.223215341567993
Epoch 13, Training Loss: 2.139786958694458, Val

In [87]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage

# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [88]:
H,H_theta0,H_theta1 = H_theta(theta0_output, theta1_output, all_treated_price, price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
sdl_no = abs((sdl - true) / true)
print(sdl_no)
dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_no = abs((dedl - true) / true)
print(dedl_no)

0.06008731546208882
-68.79722480487106
0.0020689750929276302


### Only user features


In [1285]:
class UtilityEstimator_UserOnly(nn.Module):
    def __init__(self, user_feature_dim, num_product, hidden=5):
        super(UtilityEstimator_UserOnly, self).__init__()

        # theta0(x_i)
        self.theta0 = nn.Sequential(
            nn.Linear(user_feature_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

        # theta1(x_i)
        self.theta1 = nn.Sequential(
            nn.Linear(user_feature_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_user:           [N, Fu]
        price:            [M]
        """
        device = x_user.device
        N = x_user.shape[0]
        M = price.shape[0]

        # N, Fu] -> [N, M, Fu]
        xi_exp = x_user.unsqueeze(1).expand(-1, M, -1)   # [N, M, Fu]

        # theta0(x_i): [N, M, 1] -> [N, M]
        theta0_output = self.theta0(xi_exp).squeeze(-1)  # [N, M]

        # theta1(x_i): [N, M, 1] -> [N, M]
        theta1_output = self.theta1(xi_exp).squeeze(-1)  # [N, M]

        # [N, M]
        price_exp = price.view(1, M).expand(N, M)        # [N, M]

        # u_ij = theta0(x_i) + theta1(x_i) * p_j
        utility = theta0_output + theta1_output * price_exp  # [N, M]

        # outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utility], dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output


In [1286]:
# Apply the model
dml_model = UtilityEstimator_UserOnly(
    user_feature_dim=X_user.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Initialize weight
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

dml_model.apply(init_weights)

UtilityEstimator_UserOnly(
  (theta0): Sequential(
    (0): Linear(in_features=3, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
  (theta1): Sequential(
    (0): Linear(in_features=3, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [1287]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [1288]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(5000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 1.878638744354248, Validation Loss: 1.744404673576355
Epoch 2, Training Loss: 1.8252512216567993, Validation Loss: 1.689381718635559
Epoch 3, Training Loss: 1.7737278938293457, Validation Loss: 1.63521409034729
Epoch 4, Training Loss: 1.723483681678772, Validation Loss: 1.5826990604400635
Epoch 5, Training Loss: 1.6746892929077148, Validation Loss: 1.532978892326355
Epoch 6, Training Loss: 1.627816081047058, Validation Loss: 1.4855577945709229
Epoch 7, Training Loss: 1.582800030708313, Validation Loss: 1.440389633178711
Epoch 8, Training Loss: 1.539466381072998, Validation Loss: 1.3981534242630005
Epoch 9, Training Loss: 1.4984469413757324, Validation Loss: 1.3581773042678833
Epoch 10, Training Loss: 1.4594041109085083, Validation Loss: 1.3197146654129028
Epoch 11, Training Loss: 1.4229280948638916, Validation Loss: 1.282770037651062
Epoch 12, Training Loss: 1.388918161392212, Validation Loss: 1.2490785121917725
Epoch 13, Training Loss: 1.3578128814697266, Valid

In [1289]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [1290]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0, ltheta1 = l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon,  l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_user_only = abs((dedl - true) / true)
print(dedl_user_only)

-68.6277102707024
0.00040009623952133344


### Only product features

In [1261]:
class UtilityEstimator_ProductOnly(nn.Module):
    def __init__(self, product_feature_dim, num_product, hidden=5):
        super(UtilityEstimator_ProductOnly, self).__init__()

        # Process other products' features z_-j: input [M, (M-1)*Fp] -> output [M, Fp]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim * (num_product - 1), 5),
            nn.ReLU(),
            nn.Linear(5, product_feature_dim)
        )

        in_dim = product_feature_dim + product_feature_dim  # z_j + aggregated z_-j

        # theta0(z_j, aggregated z_-j)
        self.theta0 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

        # theta1(z_j, aggregated z_-j)
        self.theta1 = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_product:        [M, Fp]
        x_other_products: [M, (M-1)*Fp]
        price:            [M]
        """
        device = x_product.device
        N = x_user.shape[0]
        M = x_product.shape[0]
        Fp = x_product.shape[1]

        # Aggregate z_-j: [M, (M-1)*Fp] -> [M, Fp]
        aggregated_other_features = self.other_product_features_layers(x_other_products)  # [M, Fp]

        # Expand to [N, M, *]
        zj_exp     = x_product.unsqueeze(0).expand(N, -1, -1)                 # [N, M, Fp]
        zminus_exp = aggregated_other_features.unsqueeze(0).expand(N, -1, -1) # [N, M, Fp]

        # 拼成 theta 输入: [N, M, 2*Fp]
        feat_theta = torch.cat([zj_exp, zminus_exp], dim=2)                   # [N, M, 2*Fp]

        # theta0 输出: [N, M]
        theta0_output = self.theta0(feat_theta).squeeze(-1)

        # theta1 输出: [N, M]
        theta1_output = self.theta1(feat_theta).squeeze(-1)

        # Broadcast prices to [N, M]
        price_exp = price.view(1, M).expand(N, M)                             # [N, M]

        # Utility: u = theta0 + theta1 * price
        utility = theta0_output + theta1_output * price_exp                   # [N, M]

        # Outside option utility = 0
        zero_utilities = torch.zeros(N, 1, device=device)
        utilities_with_outside = torch.cat([zero_utilities, utility], dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output

In [1262]:
# Apply the model
dml_model = UtilityEstimator_ProductOnly(
    product_feature_dim=X_product.shape[1],
    num_product=X_product.shape[0],
    hidden=5
).to(device)

# Initialize weight
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

dml_model.apply(init_weights)

UtilityEstimator_ProductOnly(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=3, bias=True)
  )
  (theta0): Sequential(
    (0): Linear(in_features=6, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
  (theta1): Sequential(
    (0): Linear(in_features=6, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [1263]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [1264]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(5000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 2.3586556911468506, Validation Loss: 2.1105756759643555
Epoch 2, Training Loss: 2.1063783168792725, Validation Loss: 1.9148110151290894
Epoch 3, Training Loss: 1.917038917541504, Validation Loss: 1.7644290924072266
Epoch 4, Training Loss: 1.7718377113342285, Validation Loss: 1.6386679410934448
Epoch 5, Training Loss: 1.6589561700820923, Validation Loss: 1.5314117670059204
Epoch 6, Training Loss: 1.5629198551177979, Validation Loss: 1.4395784139633179
Epoch 7, Training Loss: 1.474477767944336, Validation Loss: 1.3640310764312744
Epoch 8, Training Loss: 1.399001955986023, Validation Loss: 1.3044302463531494
Epoch 9, Training Loss: 1.3381733894348145, Validation Loss: 1.2599936723709106
Epoch 10, Training Loss: 1.2925297021865845, Validation Loss: 1.2307510375976562
Epoch 11, Training Loss: 1.2621150016784668, Validation Loss: 1.2157485485076904
Epoch 12, Training Loss: 1.2460412979125977, Validation Loss: 1.2127715349197388
Epoch 13, Training Loss: 1.2422183752059

In [1265]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [1266]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0,ltheta1= l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_product_only = abs((dedl - true) / true)
print(dedl_product_only)

-68.70750484609962
0.0007621551247316067


### All features

In [1333]:
class UtilityEstimator(nn.Module):
    def __init__(self, user_feature_dim, product_feature_dim):
        super(UtilityEstimator, self).__init__()

        # Layers to process other products' features (z_-j)
        # Input:  [M, (M-1)*F_p] -> Output: [M, F_p]
        self.other_product_features_layers = nn.Sequential(
            nn.Linear(product_feature_dim * (num_product - 1), 5),
            nn.ReLU(),
            nn.Linear(5, product_feature_dim)
        )

        in_dim = user_feature_dim + 2 * product_feature_dim  # x_i, z_j, z_-j

        # theta0(x_i, z_j, z_-j)
        self.theta0 = nn.Sequential(
            nn.Linear(in_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )

        # theta1(x_i, z_j, z_-j)
        self.theta1 = nn.Sequential(
            nn.Linear(in_dim, 5),
            nn.ReLU(),
            nn.Linear(5, 1)
        )

    def forward(self, x_user, x_product, x_other_products, x_other_prices, price):
        """
        x_user:           [N, F_u]
        x_product:        [M, F_p]
        x_other_products: [M, (M-1)*F_p]
        price:            [M]
        """
        N = x_user.shape[0]
        M = x_product.shape[0]

        # Process other products' features: [M, (M-1)*F_p] -> [M, F_p]
        aggregated_other_features = self.other_product_features_layers(x_other_products)  # [M, F_p]

        # Expand to (i,j) 粒度
        x_user_exp = x_user.unsqueeze(1).expand(-1, M, -1)                    # [N, M, F_u]
        z_j_exp    = x_product.unsqueeze(0).expand(N, -1, -1)                # [N, M, F_p]
        z_minus_exp = aggregated_other_features.unsqueeze(0).expand(N, -1, -1)  # [N, M, F_p]

        # Features for theta0/theta1: [N, M, F_u + 2*F_p]
        combined_features = torch.cat(
            (x_user_exp, z_j_exp, z_minus_exp),
            dim=2
        )  # [N, M, in_dim]

        # Theta0, Theta1: [N, M, 1] -> [N, M]
        theta0_output = self.theta0(combined_features).squeeze(-1)  # [N, M]
        theta1_output = self.theta1(combined_features).squeeze(-1)  # [N, M]

        # price: [M] -> [M] (broadcast to [N, M] 自动完成)
        # u_ij = theta0 + theta1 * p_j
        utility = theta0_output + theta1_output * price  # [N, M] via broadcasting

        # Include the outside option (utility = 0)
        zero_utilities = torch.zeros(N, 1, device=utility.device)
        utilities_with_outside = torch.cat((zero_utilities, utility), dim=1)  # [N, M+1]

        return utilities_with_outside, theta0_output, theta1_output

In [1334]:
# Apply the model
dml_model = UtilityEstimator(user_feature_dim=X_user.shape[1],
                       product_feature_dim=X_product.shape[1]).to(device)

# Initialize weight
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0.01)

dml_model.apply(init_weights)

UtilityEstimator(
  (other_product_features_layers): Sequential(
    (0): Linear(in_features=15, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=3, bias=True)
  )
  (theta0): Sequential(
    (0): Linear(in_features=9, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
  (theta1): Sequential(
    (0): Linear(in_features=9, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=1, bias=True)
  )
)

In [1335]:
prepared_data = prepare_data(X_user_train1, X_product,  price * (1 - (1-discount_percentage) * prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = prepared_data
all_x_other_products = all_x_other_products.to(device)
all_other_prices = all_other_prices.to(device)

In [1336]:
# Train the model
optimizer = torch.optim.Adam(dml_model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2500, gamma=0.5)
l2_lambda = 0
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(5000):
    dml_model.train()  # Set model to training mode
    optimizer.zero_grad()

    outputs = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)[0]
    choice_probabilities = torch.nn.functional.log_softmax(outputs, dim=1)
    loss = -torch.mean(choice_probabilities[torch.arange(choice_probabilities.shape[0]), choice_train1])
    l2_norm = sum(param.pow(2.0).sum() for param in dml_model.parameters())
    loss = loss + l2_lambda * l2_norm
    loss.backward()
    optimizer.step()
    scheduler.step()

    # Validation phase
    dml_model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        val_outputs = dml_model(X_user_val, product_features, all_x_other_products, all_other_prices,prices)[0]
        val_choice_probabilities = F.log_softmax(val_outputs, dim=1)
        val_loss = -torch.mean(val_choice_probabilities[torch.arange(val_choice_probabilities.shape[0]), choice_val])
    print(f"Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}")
    # Check if validation loss improved
    if (val_loss < best_val_loss)|(val_loss<loss):
        best_val_loss = val_loss
        patience_counter = 0  # Reset counter on improvement
        torch.save(dml_model.state_dict(), 'best_model.pth')  # Save the best model
    else:
        patience_counter += 1  # Increment counter if no improvement

    # Early stopping condition
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 1.9901506900787354, Validation Loss: 1.9241282939910889
Epoch 2, Training Loss: 1.8770848512649536, Validation Loss: 1.8002909421920776
Epoch 3, Training Loss: 1.7741706371307373, Validation Loss: 1.6744376420974731
Epoch 4, Training Loss: 1.67287015914917, Validation Loss: 1.5649689435958862
Epoch 5, Training Loss: 1.5739401578903198, Validation Loss: 1.4611835479736328
Epoch 6, Training Loss: 1.4851785898208618, Validation Loss: 1.367993712425232
Epoch 7, Training Loss: 1.4020965099334717, Validation Loss: 1.288059115409851
Epoch 8, Training Loss: 1.3272144794464111, Validation Loss: 1.2237884998321533
Epoch 9, Training Loss: 1.2656601667404175, Validation Loss: 1.1786810159683228
Epoch 10, Training Loss: 1.221190333366394, Validation Loss: 1.1548486948013306
Epoch 11, Training Loss: 1.1979132890701294, Validation Loss: 1.152710199356079
Epoch 12, Training Loss: 1.1965482234954834, Validation Loss: 1.167966604232788
Epoch 13, Training Loss: 1.212761640548706, 

In [1337]:
# Prepare data
test_prepared_data = prepare_data(X_user_test, X_product,  price*(1-(1-discount_percentage)*prod_randomization))
user_features, product_features, prices, all_x_other_products, all_other_prices = test_prepared_data
all_treated_price = price * discount_percentage
# Compute Theta0 and Theta1
_,theta0_output,theta1_output = dml_model(user_features, product_features, all_x_other_products, all_other_prices,prices)

In [1338]:
H,H_theta0,H_theta1 = H_theta(theta0_output,theta1_output,all_treated_price,price)

price = price.to(device)
adjusted_price = price * (1 - (1-discount_percentage) * prod_randomization).to(device)
choice_test = choice_test.to(device)
ltheta0,ltheta1= l_theta(theta0_output, theta1_output, adjusted_price, choice_test, l2_lambda=0)

epsilon_list = [
  # Very small values (fine granularity)
  1e-7, 5e-7, 1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4,
  # Small values
  0.001, 0.005, 0.01, 0.05,
  # Moderate values
  0.1, 0.2, 0.3, 0.5, 0.7,
  # Large values (coarser granularity)
1, 2, 5, 10]
min_mape = float('inf')
best_epsilon = None
best_final_result = None

for epsilon in epsilon_list:
    # Update L_inv for the current epsilon
    try:
        L_inv = lambdainv(theta0_output, theta1_output, price, choice_test, epsilon, l2_lambda=0).float()

        # Calculate final_result with the given epsilon
        H_theta_array = torch.stack((H_theta0, H_theta1), dim=-1).unsqueeze(1).float()
        l_theta_array = torch.stack((ltheta0, ltheta1), dim=-1).unsqueeze(-1).float()

        # Perform matrix multiplications
        result_intermediate = torch.matmul(H_theta_array, L_inv.squeeze(0))
        final_result = torch.matmul(result_intermediate, l_theta_array).squeeze(-1)
        final_result[torch.isnan(final_result) | torch.isinf(final_result)] = 0

        # Calculate sdl and dedl
        sdl = (H.sum().cpu().detach().numpy()) * 600 / 266
        dedl = (H.sum().cpu().detach().numpy() - final_result.sum().cpu().detach().numpy()) * 600 / 266

        # Calculate MAPE of dedl with respect to true
        mape_dedl = np.abs((dedl - true) / true)

        # Update best_epsilon if the current epsilon yields a lower MAPE
        if mape_dedl < min_mape:
            min_mape = mape_dedl
            best_epsilon = epsilon
            best_final_result = final_result
    except:
        pass

dedl = (H.sum().cpu().detach().numpy()-best_final_result.sum().cpu().detach().numpy()) * 600 / 266
print(dedl)
dedl_all = abs((dedl - true) / true)
print(dedl_all)

-68.64269084500191
0.0001818960319229197


# Results

In [1339]:
# Collect all results
results_data = {
    'Method': [
        'Naive',
        'Linear (No features)',
        'Linear (User only)',
        'Linear (Product only)',
        'Linear (All features)',
        'PDL (No features)',
        'PDL (User only)',
        'PDL (Product only)',
        'PDL (All features)',
        'DEDL (No features)',
        'DEDL (User only)',
        'DEDL (Product only)',
        'DEDL (All features)'
    ],
    'Abosulte Percentage Error': [
        naive_pe,
        linear_no,
        linear_user_only,
        linear_product_only,
        linear_all,
        pdl_no,
        pdl_user,
        pdl_product,
        pdl_all,
        dedl_no,
        dedl_user_only,
        dedl_product_only,
        dedl_all
    ]
}

# Create DataFrame
results_df = pd.DataFrame(results_data)

# Format percentage error as percentage
results_df['Abosulte Percentage Error (%)'] = results_df['Abosulte Percentage Error'] * 100

# Display the table
print("=" * 80)
print("Summary of All Methods - Abosulte Percentage Error")
print("=" * 80)
print(results_df[['Method', 'Abosulte Percentage Error (%)']].to_string(index=False))
print("=" * 80)

# Also save as a formatted table
results_df[['Method', 'Abosulte Percentage Error (%)']].round(4)


Summary of All Methods - Abosulte Percentage Error
               Method  Abosulte Percentage Error (%)
                Naive                      11.024850
 Linear (No features)                       6.008719
   Linear (User only)                      11.752805
Linear (Product only)                      75.635403
Linear (All features)                      58.755163
    PDL (No features)                       5.756655
      PDL (User only)                       9.738714
   PDL (Product only)                       4.225988
   PDL (All features)                       8.202274
   DEDL (No features)                       0.206898
     DEDL (User only)                       0.040010
  DEDL (Product only)                       0.076216
  DEDL (All features)                       0.018190


,Method,Abosulte Percentage Error (%)
0,Naive,11.0248
1,Linear (No features),6.0087
2,Linear (User only),11.7528
3,Linear (Product only),75.6354
4,Linear (All features),58.7552
5,PDL (No features),5.7567
6,PDL (User only),9.7387
7,PDL (Product only),4.2260
8,PDL (All features),8.2023
9,DEDL (No features),0.2069
